# Content Chunk Ingestion
This notebook works in BOTH Databricks and local Jupyter - NO CONFIGURATION NEEDED!

## Workflow
1. **Load Content**: Load content metadata from `content_repo` table
2. **Read Markdown**: Read markdown files from disk
3. **Chunk Content**: Split content using HybridMarkdownChunker (header-aware + recursive splitting)
   - Chunk size: 3000 characters (~750 tokens)
   - Overlap: 600 characters (20% for context continuity)
   - Method: First splits by headers (H1, H2, H3), then applies recursive splitting to large sections
4. **Compute Checksums**: Calculate SHA-256 hash for each chunk (automatic via chunker)
5. **Extract Metadata**: Capture header hierarchy, chunk method, and position information
6. **Ingest to Database**: Insert into `content_chunks` table with FK to `content_repo`

## Prerequisites
- Run 0_database_setup.ipynb first to create tables
- Run 1_content_repo_ingestion.ipynb to populate content_repo

## Setup

In [14]:
# Install required packages (works in both environments)
%pip install loguru pandas -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Restart Python (Databricks only - does nothing in local Jupyter)
try:
    dbutils.library.restartPython()
except NameError:
    print("✓ Local environment - no restart needed")

✓ Local environment - no restart needed


In [16]:
# CONFIGURATION - Using Repository Pattern (Phase 5)

from utility.environment import is_databricks
from database.config import DatabaseConfig
from database.backends.factory import BackendFactory
from database.repository import ContentRepository

if is_databricks():
    # Databricks: Setup widgets
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    # Databricks paths
    DATA_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update/data_ingestion/data"
    MARKDOWN_DIR = f"{DATA_PATH}/markdown"
    
    # Create database configuration
    config = DatabaseConfig(backend='databricks', catalog=CATALOG, schema=SCHEMA)
    
    print(f"📊 Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"📂 Data path: {DATA_PATH}")
    print(f"📁 Markdown directory: {MARKDOWN_DIR}")
else:
    # Local environment
    DATA_PATH = "data_ingestion/data"
    MARKDOWN_DIR = "data_ingestion/data/markdown"
    DB_PATH = "databases/faq_update.db"  # Must match 0_database_setup.ipynb
    
    # Create database configuration
    config = DatabaseConfig(backend='sqlite', db_path=DB_PATH)
    
    print(f"💾 Local mode")
    print(f"🗄️  Database: {DB_PATH}")
    print(f"📂 Data path: {DATA_PATH}")
    print(f"📁 Markdown directory: {MARKDOWN_DIR}")

print()
print(f"⚙️  Backend: {config.backend}")

# Create backend and repository
backend = BackendFactory.create_backend(config)
content_repo = ContentRepository(backend)

print(f"✓ ContentRepository initialized")
print()

2025-11-02 14:42:09.327 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton


💾 Local mode
🗄️  Database: databases/faq_update.db
📂 Data path: data_ingestion/data
📁 Markdown directory: data_ingestion/data/markdown

⚙️  Backend: sqlite
✓ ContentRepository initialized



In [17]:
# Chunking configuration - Using constants from config module
from config.constants import DEFAULT_CHUNK_SIZE, DEFAULT_CHUNK_OVERLAP

CHUNK_SIZE = DEFAULT_CHUNK_SIZE      # 3000 characters (~750 tokens)
CHUNK_OVERLAP = DEFAULT_CHUNK_OVERLAP  # 600 characters (20% overlap)

print(f"Chunk size: {CHUNK_SIZE} characters (~{CHUNK_SIZE // 4} tokens)")
print(f"Chunk overlap: {CHUNK_OVERLAP} characters ({int(CHUNK_OVERLAP / CHUNK_SIZE * 100)}% overlap)")
print(f"Effective new content per chunk: {CHUNK_SIZE - CHUNK_OVERLAP} characters")

Chunk size: 3000 characters (~750 tokens)
Chunk overlap: 600 characters (20% overlap)
Effective new content per chunk: 2400 characters


## Step 1: Load Content Repository Data
Load content metadata from `content_repo` table to get list of files to process.

In [18]:
print(f"{'='*80}")
print("  LOADING CONTENT REPOSITORY DATA - Repository Pattern")
print(f"{'='*80}\n")

# Use repository to get all active files
files = content_repo.list_files(status="Active", order_by="raw_file_nme ASC")

print(f"✓ Loaded {len(files)} active content files from repository\n")

# Convert to pandas DataFrame for easier processing
import pandas as pd
content_pdf = pd.DataFrame(files)

if not content_pdf.empty:
    print(content_pdf[['ud_source_file_id', 'raw_file_nme', 'title_nme', 'file_status']].head())
else:
    print("⚠️  No active files found!")
    print("   Please run 1_content_repo_ingestion.ipynb first.")
    raise Exception("No files to process")

print(f"\n✓ Content repository data loaded")

2025-11-02 14:42:09.364 | DEBUG    | database.repository.content_repository:list_files:345 - Listing files (status=Active, type=None, limit=None, offset=None)
2025-11-02 14:42:09.372 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 14 rows
2025-11-02 14:42:09.375 | DEBUG    | database.repository.base:execute_query:590 - Query returned 14 rows
2025-11-02 14:42:09.376 | DEBUG    | database.repository.content_repository:list_files:382 - Found 14 files


  LOADING CONTENT REPOSITORY DATA - Repository Pattern

✓ Loaded 14 active content files from repository

   ud_source_file_id               raw_file_nme  \
0                  4         Benefits_Guide.pdf   
1                  6        Code_of_Conduct.pdf   
2                 13           Email_Policy.pdf   
3                  1      Employee_Handbook.pdf   
4                  8  Expense_Reimbursement.pdf   

                                           title_nme file_status  
0      Employee Benefits Overview - Health Insurance      Active  
1            Code of Conduct - Ethics and Compliance      Active  
2                                 Email Usage Policy      Active  
3                             Employee Handbook 2024      Active  
4  Expense Reimbursement Policy - Submission Guid...      Active  

✓ Content repository data loaded


---
## Step 2: Chunk Content and Compute Checksums

For each file:
1. Read markdown content from disk
2. Split using **HybridMarkdownChunker** (production-recommended strategy):
   - **First pass**: Split by markdown headers (H1, H2, H3) to preserve document structure
   - **Second pass**: Apply recursive splitting to large sections for size control
   - **Overlap**: 600 characters (20%) between chunks for context continuity
3. Compute SHA-256 checksum for each chunk (automatic)
4. Extract rich metadata: header hierarchy, chunk method, position information

In [19]:
print(f"\n{'='*80}")
print("  CHUNKING AND CHECKSUM COMPUTATION - HybridMarkdownChunker")
print(f"{'='*80}\n")

# Import required modules
from chunking import HybridMarkdownChunker
from data_ingestion import ChunkIngestion
from pathlib import Path

# Initialize chunker with production-recommended settings
chunker = HybridMarkdownChunker(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Initialize chunk ingestion (uses same repository)
chunk_ingestion = ChunkIngestion(repository=content_repo)

# Process each file
files_processed = 0
total_chunks = 0
ingestion_results = []

for idx, row in content_pdf.iterrows():
    # Build path to markdown file
    if is_databricks():
        # Databricks: Use absolute path from volume
        markdown_path = Path(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update") / row['extracted_markdown_file_path']
    else:
        # Local: Use relative path
        markdown_path = Path(row['extracted_markdown_file_path'])
    
    if not markdown_path.exists():
        print(f"⚠️  WARNING: File not found: {markdown_path}")
        continue
    
    # Read content
    try:
        content = markdown_path.read_text(encoding='utf-8')
    except Exception as e:
        print(f"✗ ERROR reading {markdown_path}: {e}")
        continue
    
    # Chunk content with automatic checksum computation and metadata extraction
    # Returns: List of (chunk_text, checksum, ChunkMetadata) tuples
    try:
        chunks_with_checksums = chunker.chunk_with_checksums(content)
    except Exception as e:
        print(f"✗ ERROR chunking {row['raw_file_nme']}: {e}")
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': False,
            'chunks': 0,
            'error': f"Chunking error: {str(e)}"
        })
        continue
    
    # Ingest chunks using ChunkIngestion (handles clearing + insertion)
    try:
        result = chunk_ingestion.ingest_chunks_for_file(
            ud_source_file_id=row['ud_source_file_id'],
            chunks_with_checksums=chunks_with_checksums,
            clear_existing=True  # Clear old chunks before inserting new ones
        )
        
        if result.get('success'):
            files_processed += 1
            rows_inserted = result.get('rows_inserted', 0)
            total_chunks += rows_inserted
            print(f"  ✓ {row['raw_file_nme']:40s} → {rows_inserted:3d} chunks")
        else:
            print(f"  ✗ {row['raw_file_nme']:40s} → ERROR: {result.get('message', 'Unknown error')}")
        
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': result.get('success', False),
            'chunks': rows_inserted if result.get('success') else 0,
            'error': result.get('message', '') if not result.get('success') else None
        })
    except Exception as e:
        print(f"  ✗ {row['raw_file_nme']:40s} → ERROR: {str(e)}")
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': False,
            'chunks': 0,
            'error': str(e)
        })

print(f"\n✓ Processed {files_processed} files")
print(f"✓ Generated {total_chunks} chunks")
print(f"✓ Average chunks per file: {total_chunks / files_processed if files_processed > 0 else 0:.1f}")


  CHUNKING AND CHECKSUM COMPUTATION - HybridMarkdownChunker



2025-11-02 14:42:09.457 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.462 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.463 | INFO     | database.repository.content_repository:clear_chunks_for_file:1282 - Clearing all chunks for file ID: 4
2025-11-02 14:42:09.464 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:09.465 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:09.466 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected 0 rows
2025-11-02 14:42:09.467 | DEBUG    | database.repository.base:execute_command:682 - Command affected 0 rows
2025-11-02 14:42:09.468 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:09.469 | DEBUG    | database.repository.base:transaction:371 - Transaction committed

  ✓ Benefits_Guide.pdf                       →   1 chunks


2025-11-02 14:42:09.500 | INFO     | database.repository.content_repository:ingest_chunks:1159 - Ingesting 1 chunks in bulk
2025-11-02 14:42:09.501 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:09.501 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:09.504 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 1 rows into content_chunks
2025-11-02 14:42:09.504 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:09.505 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:09.506 | INFO     | database.repository.content_repository:ingest_chunks:1240 - Successfully ingested 1 chunks
2025-11-02 14:42:09.508 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.508 | DEBUG    | database.repository.base:execute_que

  ✓ Code_of_Conduct.pdf                      →   1 chunks
  ✓ Email_Policy.pdf                         →   1 chunks
  ✓ Employee_Handbook.pdf                    →   1 chunks


...

  ✓ Performance_Reviews.pdf                  →   1 chunks


2025-11-02 14:42:09.669 | INFO     | database.repository.content_repository:ingest_chunks:1240 - Successfully ingested 1 chunks
2025-11-02 14:42:09.673 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.675 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.677 | INFO     | database.repository.content_repository:clear_chunks_for_file:1282 - Clearing all chunks for file ID: 7
2025-11-02 14:42:09.679 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:09.679 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:09.681 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected 0 rows
2025-11-02 14:42:09.683 | DEBUG    | database.repository.base:execute_command:682 - Command affected 0 rows
2025-11-02 14:42:09.685 | DEBUG    | database.backends.sqlite.backend:commit:5

  ✓ Remote_Work_Policy.pdf                   →   1 chunks


2025-11-02 14:42:09.685 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:09.687 | INFO     | database.repository.content_repository:clear_chunks_for_file:1290 - Deleted 0 chunks for file 7
2025-11-02 14:42:09.691 | INFO     | database.repository.content_repository:ingest_chunks:1159 - Ingesting 1 chunks in bulk
2025-11-02 14:42:09.693 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:09.695 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:09.698 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 1 rows into content_chunks
2025-11-02 14:42:09.700 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:09.701 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:09.701 | INFO     | database.repository.content_

  ✓ Travel_Policy.pdf                        →   1 chunks
⚠️  WARNING: File not found: data_ingestion\data\markdown\Vacation_Policy_v1.md

✓ Processed 12 files
✓ Generated 12 chunks
✓ Average chunks per file: 1.0


In [20]:
# Summary of ingestion results
import pandas as pd

results_df = pd.DataFrame(ingestion_results)
print(f"\nIngestion Results Summary:")
print(results_df.to_string(index=False))
print(f"\nData Summary:")
print(f"  Files processed: {results_df['success'].sum()}")
print(f"  Total chunks: {results_df['chunks'].sum()}")
print(f"  Failed files: {(~results_df['success']).sum()}")


Ingestion Results Summary:
 file_id                 file_name  success  chunks error
       4        Benefits_Guide.pdf     True       1  None
       6       Code_of_Conduct.pdf     True       1  None
      13          Email_Policy.pdf     True       1  None
       1     Employee_Handbook.pdf     True       1  None
       8 Expense_Reimbursement.pdf     True       1  None
      11      Health_Insurance.pdf     True       1  None
       2    IT_Security_Policy.pdf     True       1  None
      14        Parking_Policy.pdf     True       1  None
       5    Payroll_Processing.pdf     True       1  None
       9   Performance_Reviews.pdf     True       1  None
       3    Remote_Work_Policy.pdf     True       1  None
       7         Travel_Policy.pdf     True       1  None

Data Summary:
  Files processed: 12
  Total chunks: 12
  Failed files: 0


---
## Step 3: Verify Ingestion

Verify the ingested chunks with sample queries.

In [21]:
print(f"\n{'='*80}")
print("  DATABASE VERIFICATION - Repository Pattern")
print(f"{'='*80}\n")

# Get statistics using repository
stats = content_repo.get_chunk_stats()

print("📊 Chunk Statistics:")
print(f"  Total chunks: {stats.get('total_chunks', 0)}")
print(f"  Unique checksums: {stats.get('unique_checksums', 0)}")
print(f"  Average chunks per file: {stats.get('avg_chunks_per_file', 0)}")
print()
print(f"  By Status:")
for status, count in stats.get('by_status', {}).items():
    print(f"    {status}: {count}")

2025-11-02 14:42:09.742 | DEBUG    | database.repository.content_repository:get_chunk_stats:1327 - Calculating chunk statistics (file_id=None)
2025-11-02 14:42:09.743 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.744 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.745 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.746 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.748 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.750 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.751 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.751 | DEBUG    | database.repository.base:execute_query:590 - Query returned 


  DATABASE VERIFICATION - Repository Pattern

📊 Chunk Statistics:
  Total chunks: 12
  Unique checksums: 12
  Average chunks per file: 1.0

  By Status:
    active: 12


In [22]:
# No longer needed - removed this cell

---
## Verification Queries

Query the database to verify ingested chunks.

In [23]:
print(f"\n{'='*80}")
print("  VERIFICATION QUERIES - Repository Pattern")
print(f"{'='*80}\n")

# Query 1: Sample chunks with metadata (first 5)
print("Query 1: Sample chunks with rich metadata\n")

# Get first file and show its chunks
if not content_pdf.empty:
    first_file_id = content_pdf.iloc[0]['ud_source_file_id']
    first_file_name = content_pdf.iloc[0]['raw_file_nme']
    
    chunks = content_repo.get_chunks_by_file(first_file_id)
    
    print(f"Chunks from {first_file_name} (file_id={first_file_id}):")
    print(f"Total chunks: {len(chunks)}\n")
    
    for i, chunk in enumerate(chunks[:5]):
        chunk_text = chunk['chunk_text']
        chunk_len = len(chunk_text)
        
        print(f"  Chunk {chunk['chunk_index']}:")
        print(f"    Checksum: {chunk['content_checksum'][:16]}...")
        print(f"    Size: {chunk_len:,} characters (~{chunk_len // 4} tokens)")
        
        # Display metadata if available
        if chunk.get('chunk_method'):
            print(f"    Method: {chunk['chunk_method']}")
        
        if chunk.get('chunk_headers'):
            # Parse JSON headers
            headers = content_repo.parse_chunk_headers(chunk['chunk_headers'])
            if headers:
                headers_str = " > ".join([f"{k.upper()}: {v}" for k, v in headers.items()])
                print(f"    Headers: {headers_str}")
        
        if chunk.get('chunk_start_char') is not None and chunk.get('chunk_end_char') is not None:
            print(f"    Position: chars {chunk['chunk_start_char']:,} - {chunk['chunk_end_char']:,}")
        
        print(f"    Text preview: {chunk_text[:100].replace(chr(10), ' ')}...")
        
        # Show overlap with next chunk if available
        if i < len(chunks[:5]) - 1 and i + 1 < len(chunks):
            next_chunk = chunks[i + 1]
            next_text = next_chunk['chunk_text']
            
            # Find overlap by checking end of current with start of next
            max_overlap = min(len(chunk_text), len(next_text))
            overlap_size = 0
            for j in range(1, max_overlap + 1):
                if chunk_text[-j:] == next_text[:j]:
                    overlap_size = j
            
            if overlap_size > 0:
                overlap_pct = (overlap_size / chunk_len) * 100
                print(f"    Overlap with next: {overlap_size} chars ({overlap_pct:.1f}%)")
        
        print()
else:
    print("  No files to show chunks for")

2025-11-02 14:42:09.778 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 4
2025-11-02 14:42:09.778 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.778 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.778 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 4



  VERIFICATION QUERIES - Repository Pattern

Query 1: Sample chunks with rich metadata

Chunks from Benefits_Guide.pdf (file_id=4):
Total chunks: 1

  Chunk 0:
    Checksum: e4e657579f05c0b5...
    Size: 350 characters (~87 tokens)
    Method: no_split
    Position: chars 0 - 350
    Text preview: # Employee Benefits Overview - Health Insurance  ## Health Insurance Plans  Three plan options avail...



In [24]:
# Query 2: Chunks per file with size and metadata statistics
print("\n\nQuery 2: Chunks per source file with metadata analysis\n")

# Get chunk counts and statistics for each file
chunk_stats = []
for _, row in content_pdf.iterrows():
    file_id = row['ud_source_file_id']
    chunks = content_repo.get_chunks_by_file(file_id)
    
    if chunks:
        chunk_sizes = [len(c['chunk_text']) for c in chunks]
        avg_size = sum(chunk_sizes) / len(chunk_sizes) if chunk_sizes else 0
        min_size = min(chunk_sizes) if chunk_sizes else 0
        max_size = max(chunk_sizes) if chunk_sizes else 0
        
        # Count chunking methods
        methods = [c.get('chunk_method') for c in chunks if c.get('chunk_method')]
        method_summary = f"{len([m for m in methods if m == 'header'])}H/{len([m for m in methods if m == 'recursive'])}R/{len([m for m in methods if m == 'no_split'])}N" if methods else "N/A"
    else:
        avg_size = min_size = max_size = 0
        method_summary = "N/A"
    
    chunk_stats.append({
        'raw_file_nme': row['raw_file_nme'],
        'chunks': len(chunks),
        'avg_size': int(avg_size),
        'method_split': method_summary,
        'file_status': row.get('file_status', 'N/A')
    })

# Convert to DataFrame and display
import pandas as pd
result_df = pd.DataFrame(chunk_stats)
result_df = result_df.sort_values('chunks', ascending=False)

print(result_df.to_string(index=False))
print(f"\nLegend: H=Header-split, R=Recursive-split, N=No-split")

# Overall statistics
print(f"\n{'='*80}")
print("Overall Chunk Statistics:")
print(f"{'='*80}")
total_chunks = result_df['chunks'].sum()
avg_chunks_per_file = result_df['chunks'].mean()
avg_size_overall = result_df['avg_size'].mean()

print(f"Total chunks: {total_chunks}")
print(f"Average chunks per file: {avg_chunks_per_file:.1f}")
print(f"Average chunk size: {avg_size_overall:.0f} characters (~{avg_size_overall / 4:.0f} tokens)")
print(f"Target chunk size: {CHUNK_SIZE} characters (with {CHUNK_OVERLAP} char overlap)")

# Metadata quality check
chunks_with_metadata = 0
total_all_chunks = 0
for _, row in content_pdf.iterrows():
    file_chunks = content_repo.get_chunks_by_file(row['ud_source_file_id'])
    total_all_chunks += len(file_chunks)
    chunks_with_metadata += len([c for c in file_chunks if c.get('chunk_method')])

if total_all_chunks > 0:
    metadata_pct = (chunks_with_metadata / total_all_chunks) * 100
    print(f"\nMetadata Coverage:")
    print(f"  Chunks with metadata: {chunks_with_metadata}/{total_all_chunks} ({metadata_pct:.1f}%)")

2025-11-02 14:42:09.803 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 4
2025-11-02 14:42:09.805 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.806 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.807 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 4
2025-11-02 14:42:09.807 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 6
2025-11-02 14:42:09.809 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.810 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.811 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 6
2025-11-02 14:42:09.813 | DEBUG    | datab



Query 2: Chunks per source file with metadata analysis



2025-11-02 14:42:09.826 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 2
2025-11-02 14:42:09.827 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 10
2025-11-02 14:42:09.828 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:09.829 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:09.830 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 0 chunks for file 10
2025-11-02 14:42:09.830 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 14
2025-11-02 14:42:09.831 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.832 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.832 | DEBUG    | da

             raw_file_nme  chunks  avg_size method_split file_status
       Benefits_Guide.pdf       1       350     0H/0R/1N      Active
      Code_of_Conduct.pdf       1       378     0H/0R/1N      Active
         Email_Policy.pdf       1       431     0H/0R/1N      Active
    Employee_Handbook.pdf       1       383     0H/0R/1N      Active
Expense_Reimbursement.pdf       1       383     0H/0R/1N      Active
     Health_Insurance.pdf       1       522     0H/0R/1N      Active
   IT_Security_Policy.pdf       1       296     0H/0R/1N      Active
       Parking_Policy.pdf       1       448     0H/0R/1N      Active
   Payroll_Processing.pdf       1       367     0H/0R/1N      Active
  Performance_Reviews.pdf       1       415     0H/0R/1N      Active
        Travel_Policy.pdf       1       312     0H/0R/1N      Active
   Remote_Work_Policy.pdf       1       336     0H/0R/1N      Active
       Parental_Leave.pdf       0         0          N/A      Active
      Vacation_Policy.pdf       0 

2025-11-02 14:42:09.857 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 6
2025-11-02 14:42:09.858 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.859 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.859 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 6
2025-11-02 14:42:09.860 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1018 - Retrieving chunks for file ID: 13
2025-11-02 14:42:09.861 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:09.862 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:09.862 | DEBUG    | database.repository.content_repository:get_chunks_by_file:1029 - Found 1 chunks for file 13
2025-11-02 14:42:09.862 | DEBUG    | dat


Metadata Coverage:
  Chunks with metadata: 12/12 (100.0%)


## Summary

### Completed:
- Loaded content metadata from `content_repo` table
- Read markdown files from disk
- Split content using **HybridMarkdownChunker** (production-recommended):
  - Chunk size: 3000 characters (~750 tokens)
  - Overlap: 600 characters (20% for context continuity)
  - Strategy: Header-aware splitting (H1, H2, H3) + recursive splitting for large sections
- Computed SHA-256 checksums automatically via chunker
- Extracted rich metadata: header hierarchy, chunk method, position information
- Ingested chunks into `content_chunks` table with FK to `content_repo`
- Verified data in database

### Technical Details:
- **Chunker**: `HybridMarkdownChunker` from chunking module (v3.0.0)
- **Ingestion**: `ChunkIngestion` from data_ingestion module (repository pattern)
- **Metadata**: Each chunk includes header context, chunking method, and position data
- **Checksums**: SHA-256 hashes computed automatically during chunking

### Next Steps:
1. **Generate FAQs**: Run FAQ generation notebook to create FAQs from chunks
2. **Link Sources**: FAQs will reference chunks in `faq_question_sources` and `faq_answer_sources`
3. **Change Detection**: When content changes, new chunks enable precise change tracking
4. **Impact Analysis**: Identify which FAQs are affected by specific content changes

In [25]:
print(f"\n{'='*80}")
print("  CHUNK INGESTION COMPLETE")
print(f"{'='*80}\n")
print("✓ All chunks successfully computed and ingested")
print("✓ Data verified in database")
print("✓ Ready for FAQ generation")
print(f"\n{'='*80}")


  CHUNK INGESTION COMPLETE

✓ All chunks successfully computed and ingested
✓ Data verified in database
✓ Ready for FAQ generation



In [26]:
# Import the database module
from database import get_backend

# Get a backend instance (auto-detects SQLite or Databricks based on environment)
backend = get_backend()

# Alternative: Get dataframe from content_checksums table (the newer table schema)
df_content_chunks= backend.read_table("content_chunks")

print("Content Checksums DataFrame:")
df_content_chunks

2025-11-02 14:42:09.933 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton
2025-11-02 14:42:09.939 | DEBUG    | database.backends.sqlite.backend:read_table:486 - Read 12 rows from content_chunks


Content Checksums DataFrame:


chunk_id  ud_source_file_id  chunk_index  \
0          1                  4            0   
1          2                  6            0   
2          3                 13            0   
3          4                  1            0   
4          5                  8            0   
5          6                 11            0   
6          7                  2            0   
7          8                 14            0   
8          9                  5            0   
9         10                  9            0   
10        11                  3            0   
11        12                  7            0   

                                     content_checksum  \
0   e4e657579f05c0b56e0cf9d2c3c60bcbfae818ac84cedd...   
1   a63a029055575b7aaedfa8d55dad9b859305a101b1f5e7...   
2   26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...   
3   b54c22ec37e28c72f242c5c96ba13a0d582cf14ad4bba7...   
4   21c3224cf3de0229bb0953c537dfe703be7752c5a3513e...   
5   370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...   
6   21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...   
7   9638205ff31b28380b3165acf62ba48de76e2cf45fadda...   
8   32cbadc16b6c3e6976b4158e3ffd945855495cb3ccc17d...   
9   5344d68abbaa53f9ce78175e24ce7282042d98399bc4a8...   
10  525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...   
11  c0a9947d707ea77c52566453c69cc6aef7cb3f07bcb742...   

                                           chunk_text chunk_page_number  \
0   # Employee Benefits Overview - Health Insuranc...              None   
1   # Code of Conduct - Ethics and Compliance\n\n#...              None   
2   # Email Policy\n\nCompany email should be used...              None   
3   # Employee Handbook 2024\n\n## Time Off Policy...              None   
4   # Expense Reimbursement Policy - Submission Gu...              None   
5   # Health Insurance Enrollment Policy\n\nAll fu...              None   
6   # IT Security Policy - Password Requirements\n...              None   
7   # Parking Policy\n\nEmployees may park in desi...              None   
8   # Payroll Processing Guide - Tax Withholding\n...              None   
9   # Annual Performance Review Process\n\n## Revi...              None   
10  # Remote Work Policy - Eligibility and Guideli...              None   
11  # Business Travel Policy - Domestic Travel\n\n...              None   

    chunk_start_char  chunk_end_char chunk_method chunk_headers  \
0                  0             350     no_split          None   
1                  0             378     no_split          None   
2                  0             431     no_split          None   
3                  0             383     no_split          None   
4                  0             383     no_split          None   
5                  0             522     no_split          None   
6                  0             296     no_split          None   
7                  0             448     no_split          None   
8                  0             367     no_split          None   
9                  0             415     no_split          None   
10                 0             336     no_split          None   
11                 0             312     no_split          None   

                    created_at  status  
0   2025-11-02 14:42:09.475076  active  
1   2025-11-02 14:42:09.500340  active  
2   2025-11-02 14:42:09.516792  active  
3   2025-11-02 14:42:09.528050  active  
4   2025-11-02 14:42:09.545354  active  
5   2025-11-02 14:42:09.562158  active  
6   2025-11-02 14:42:09.578805  active  
7   2025-11-02 14:42:09.593618  active  
8   2025-11-02 14:42:09.609471  active  
9   2025-11-02 14:42:09.640590  active  
10  2025-11-02 14:42:09.657351  active  
11  2025-11-02 14:42:09.691919  active